# Introduction

In this tutorial, we'll discuss one of the foundational  algorithms of machine learning in this post: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. 

#  Problem Statement
Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

In [1]:
! python -m pip -q install pip

In [2]:
! python -m pip -q install jovian
! python -m pip -q install numpy torch torchvision 

In [3]:
import torch
import numpy as np

# Training Data

In [4]:
X = np.array([
              [73, 67, 43],
              [91, 88, 43],
              [87, 134, 43],
              [102, 43, 43],
              [69, 96, 43],
], dtype='float32')

# Targets (apples, oranges)
Y = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [5]:
# convert inputs and targets to tensors

inputs = torch.from_numpy(X)
targets = torch.from_numpy(Y)
print(inputs.shape)
print(targets.shape)

torch.Size([5, 3])
torch.Size([5, 2])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [6]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(1, 2, requires_grad=True)
print(w)
print(b)

tensor([[-1.3291,  0.4082, -1.7480],
        [ 0.2926,  0.0272, -1.5048]], requires_grad=True)
tensor([[0.6521, 0.6052]], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [7]:
def LinearRegression(X):
    return X @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [8]:
preds = LinearRegression(inputs)
print(preds)

tensor([[-144.1861,  -40.9149],
        [-159.5373,  -35.0758],
        [-135.4439,  -34.9934],
        [-192.5261,  -33.0827],
        [-127.0320,  -41.2955]], grad_fn=<AddBackward0>)


In [9]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [10]:
diff = (preds - targets)
# diff.numel()
torch.sum(diff ** 2)/diff.numel()

tensor(35125.7344, grad_fn=<DivBackward0>)

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [11]:
# MSE loss

def MSE(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 178.3390 (square root of the loss 31804.8008)*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the *loss*, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

In [12]:
# compute loss

loss = MSE(preds, targets)
print(loss)

tensor(35125.7344, grad_fn=<DivBackward0>)


## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [13]:
# gradient computation

loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [14]:
print(w)
print(w.grad)

tensor([[-1.3291,  0.4082, -1.7480],
        [ 0.2926,  0.0272, -1.5048]], requires_grad=True)
tensor([[-19278.5938, -19996.5859,  -9801.6387],
        [-10660.7891, -12063.8047,  -5550.1167]])


The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [15]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0.]])


In [16]:
w, b

(tensor([[-1.3291,  0.4082, -1.7480],
         [ 0.2926,  0.0272, -1.5048]], requires_grad=True),
 tensor([[0.6521, 0.6052]], requires_grad=True))

## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [17]:
# generate predictions
preds = LinearRegression(inputs)
print(f"Predictions: {preds}")

# calculate the loss
loss = MSE(preds, targets)
print(loss)

# computer gradients
loss.backward()
print(w.grad)
print(b.grad)

# adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Predictions: tensor([[-144.1861,  -40.9149],
        [-159.5373,  -35.0758],
        [-135.4439,  -34.9934],
        [-192.5261,  -33.0827],
        [-127.0320,  -41.2955]], grad_fn=<AddBackward0>)
tensor(35125.7344, grad_fn=<DivBackward0>)
tensor([[-19278.5938, -19996.5859,  -9801.6387],
        [-10660.7891, -12063.8047,  -5550.1167]])
tensor([[-227.9451, -129.0725]])


A few things to note above:

* We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

* We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the *learning rate* of the algorithm. 

* After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

In [18]:
# Let's take a look at the new weights and biases.
print(w)
print(b)

tensor([[-1.1363,  0.6082, -1.6500],
        [ 0.3992,  0.1479, -1.4493]], requires_grad=True)
tensor([[0.6544, 0.6065]], requires_grad=True)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

In [19]:
# calculate loss

preds = LinearRegression(inputs)
loss = MSE(preds, targets)
print(loss)

tensor(24514.9102, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

In [20]:
for i in range(1000):
    preds = LinearRegression(inputs)
    loss = MSE(preds, targets)
    loss.backward()
        
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [21]:
# calculate loss

preds = LinearRegression(inputs)
loss = MSE(preds, targets)
print(loss)

tensor(126.4621, grad_fn=<DivBackward0>)


In [22]:
preds

tensor([[ 53.0666,  65.2729],
        [ 78.6047,  95.7786],
        [132.3161, 150.6583],
        [ 26.4247,  44.0407],
        [ 86.8523,  99.4645]], grad_fn=<AddBackward0>)

In [23]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [24]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [25]:
# Input (temp, rainfall, humidity)
X = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
Y = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(X)
targets = torch.from_numpy(Y)

In [26]:
inputs, targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [27]:
inputs.shape, targets.shape

(torch.Size([15, 3]), torch.Size([15, 2]))

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [28]:
from torch.utils.data import TensorDataset 

In [29]:
train_ds = TensorDataset(inputs, targets)
train_ds[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [30]:
 from torch.utils.data import DataLoader

In [31]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a `for-in` loop. Let's look at an example.

In [32]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.]])
tensor([[ 56.,  70.],
        [103., 119.],
        [119., 133.],
        [ 81., 101.],
        [ 22.,  37.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [33]:
# define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3061, -0.4487,  0.3670],
        [-0.3929,  0.5728, -0.5292]], requires_grad=True)
Parameter containing:
tensor([-0.1505, -0.0819], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [34]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3061, -0.4487,  0.3670],
         [-0.3929,  0.5728, -0.5292]], requires_grad=True),
 Parameter containing:
 tensor([-0.1505, -0.0819], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [35]:
preds = model(inputs)
print(preds)

tensor([[-36.7773, -13.1424],
        [-44.0025, -19.2996],
        [-65.6205,  11.7941],
        [-37.0876, -35.1064],
        [-38.6558,  -9.2499],
        [-36.7773, -13.1424],
        [-44.0025, -19.2996],
        [-65.6205,  11.7941],
        [-37.0876, -35.1064],
        [-38.6558,  -9.2499],
        [-36.7773, -13.1424],
        [-44.0025, -19.2996],
        [-65.6205,  11.7941],
        [-37.0876, -35.1064],
        [-38.6558,  -9.2499]], grad_fn=<AddmmBackward>)


## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [36]:
from torch.nn import functional as func

In [37]:
loss_fn = func.mse_loss 

In [38]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(13959.8574, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [39]:
from torch import optim

opt = optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

* The model's parameter gradient are calculated from the `loss.backward()` as `loss(loss_fn(model(..), y)`.
* Because of this the model's parameters gradient is calculated and stored in their method i.e in `model.weight.grad`, the optimizer uses this gradient to update the weights.

[Stack overflow](https://stackoverflow.com/a/53975741/6805747)

In [40]:
def fit(num_epochs, model, loss_function, optimizer, data_loader):

    for epoch in range(num_epochs):

        for xb, yb in data_loader:
            
            # generate predictions
            pred = model(xb)

            # calculate loss
            loss = loss_function(pred, yb)
            
            # compute gradients
            # print("before")
            # print(list(model.parameters()))
            # print(model.weight.grad)
            loss.backward()

            # update parameters using gradients
            optimizer.step()
            # print("after")
            # print(list(model.parameters()))
            # print(model.weight.grad)
            
            # reset gradients to zero
            optimizer.zero_grad()
            
            # break
        # break
    
       # Print the progress
        if (epoch+1) % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [41]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [100/1000], Loss: 51.4847
Epoch [200/1000], Loss: 32.9811
Epoch [300/1000], Loss: 8.6214
Epoch [400/1000], Loss: 10.9054
Epoch [500/1000], Loss: 6.2042
Epoch [600/1000], Loss: 3.5731
Epoch [700/1000], Loss: 2.8088
Epoch [800/1000], Loss: 1.7266
Epoch [900/1000], Loss: 1.1911
Epoch [1000/1000], Loss: 0.8487


In [42]:
preds = model(inputs)
preds

tensor([[ 57.1185,  70.3591],
        [ 82.2636, 100.2058],
        [118.5552, 133.9635],
        [ 21.0317,  37.3033],
        [102.0245, 118.1454],
        [ 57.1185,  70.3591],
        [ 82.2636, 100.2058],
        [118.5552, 133.9635],
        [ 21.0317,  37.3033],
        [102.0245, 118.1454],
        [ 57.1185,  70.3591],
        [ 82.2636, 100.2058],
        [118.5552, 133.9635],
        [ 21.0317,  37.3033],
        [102.0245, 118.1454]], grad_fn=<AddmmBackward>)

In [43]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [44]:
import jovian

jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "vaibhav-singh-3001/02-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/vaibhav-singh-3001/02-linear-regression


'https://jovian.ml/vaibhav-singh-3001/02-linear-regression'